In [1]:
import pandas as pd 
import celescope
import time
import bisect
from celescope.tools.emptydrop_cr.cell_calling_3 import cell_calling_3
from celescope.tools.count import Count

In [2]:
celescope.__path__

['/SGRNJ06/randd/USER/cjj/Celescope/refactor_trust/CeleScope/celescope']

In [3]:
# cr
df = pd.read_table('/SGRNJ06/randd/PROJECT/RD20073101_ScRNA_VDJ/20220425_kmc3/sc-1/05.count/220417014_count_detail.txt', header=0)

In [4]:
start_time = time.time()
df_sum = Count.get_df_sum(df)
end_time = time.time()
print(f'cost {end_time - start_time}')

cost 317.9189121723175


In [5]:
df_sum

,readcount,UMI2,UMI,geneID
Barcode,,,,
AACAACCAGTCGTAGACACCTTAC,210001,181881,76645,6778
GTACGCAAGCCAAGACCGAACTTA,189400,176090,52942,2487
ACTATGCAAACCGAGACCGACAAC,163141,146897,52730,3240
CCTCTATCCAATGGAAGACAGTGC,167824,153922,50412,2548
TGGAACAACCTCCTGAACGTATCA,162767,148077,50098,2794
...,...,...,...,...
TAGGATGAACTATGCAAACCGAGA,1,0,1,1
AGTCACTAGAATCTGACGACACAC,1,0,1,1
ACATTGGCCTCAATGATTCACGCA,1,0,1,1


In [12]:
    def auto_cell(df_sum):
        idx = int(3000 * 0.01)
        barcode_number = df_sum.shape[0]
        idx = int(min(barcode_number, idx))
        if idx == 0:
            sys.exit("cell number equals zero!")
        # calculate read counts threshold
        threshold = int(Count.find_threshold(df_sum, idx) * 0.1)
        threshold = max(1, threshold)
        cell_bc = Count.get_cell_bc(df_sum, threshold)

        return cell_bc, threshold

    def emptydrop_cr_cell(df_sum):
        cell_bc, initial_cell_num = cell_calling_3('/SGRNJ06/randd/PROJECT/RD20073101_ScRNA_VDJ/20220425_kmc3/sc-1/05.count/220417014_all_matrix', 3000)
        threshold = Count.find_threshold(df_sum, initial_cell_num)
        return cell_bc, threshold
    
    

In [13]:
df_sum_auto, df_sum_emptydrops = df_sum.copy(), df_sum.copy()
cell_bc_auto, threshold_auto = auto_cell(df_sum_auto)
cell_bc_emptydrops, threshold_emptydrops = emptydrop_cr_cell(df_sum_emptydrops)

2022-07-11 10:10:56,918 - celescope.tools.matrix.from_matrix_dir - INFO - start...
2022-07-11 10:11:44,006 - celescope.tools.matrix.from_matrix_dir - INFO - done. time used: 0:00:47.086092
Cell-called barcodes metrics:
filtered_bcs: 3154
filtered_bcs_var: 5262.525899999999
max_filtered_bcs: 18000.0
Median UMIs of initial cell calls: 9439.0
Min UMIs: 500
Number of candidate bcs: 5389
Range candidate bc umis: 500, 4093
Number of distinct N supplied: 1665
Range of N: 3593
Number of features: 23891
....................................................................................................
Number of non-ambient barcodes from SGT: 3065
Identify 6219 cell-associated barcodes


In [24]:
    def get_cell_stats(df_sum, cell_bc):
        df_sum.loc[:, 'mark'] = 'UB'
        df_sum.loc[df_sum.index.isin(cell_bc), 'mark'] = 'CB'
        CB_describe = df_sum.loc[df_sum['mark'] == 'CB', :].describe()
        return CB_describe

In [57]:
            CB_describe_auto = get_cell_stats(df_sum_auto, cell_bc_auto)
            median_genes_per_cell_auto = int(CB_describe_auto.loc['50%', 'geneID'])

In [58]:
            CB_describe_emptydrops = get_cell_stats(df_sum_emptydrops, cell_bc_emptydrops)
            median_genes_per_cell_emptydrops = int(CB_describe_emptydrops.loc['50%', 'geneID'])

In [103]:
            target_median_gene = median_genes_per_cell_auto * 0.8
            low = len(cell_bc_auto)
            high = len(cell_bc_emptydrops)
            while low <= high:
                mid = (low + high) // 2
                current_median_gene = df_sum_mixed.loc[df_sum_mixed['mark'] == 'CB', :][:mid].describe().loc['50%', 'geneID']
                print(mid)
                print(current_median_gene)
                if current_median_gene == target_median_gene:
                    break
                elif current_median_gene > target_median_gene:
                    low = mid + 1
                else:
                    high = mid - 1

4686
2000.5
5453
1792.0
5069
1901.0
4877
1949.0
4781
1973.0
4733
1987.0
4709
1995.0
4721
1990.0
4715
1991.0
4712
1992.0


In [114]:
            cell_bc = df_sum_mixed.loc[df_sum_mixed['mark'] == 'CB', :][:mid].reset_index().Barcode.tolist()
            threshold = Count.find_threshold(df_sum, len(cell_bc))

In [110]:
current_median_gene = df_sum_mixed.loc[df_sum_mixed['mark'] == 'CB', :][:mid].describe().loc['50%', 'geneID']

In [111]:
current_median_gene

1992.0

In [117]:
threshold

996

In [106]:
threshold_emptydrops

4098

In [107]:
threshold_auto

In [118]:
len(cell_bc)

4712

In [122]:
Count.find_threshold(df_sum, len(cell_bc_emptydrops))

653

In [ ]:
df_sum.loc[:, 'mark'] = 'UB'
df_sum.loc[df_sum.index.isin(cell_bc), 'mark'] = 'CB'
CB_describe = df_sum.loc[df_sum['mark'] == 'CB', :].describe()

In [ ]:
df_sum.loc[df_sum['mark'] == 'CB', :][:4000]
CB_describe = df_sum.loc[df_sum['mark'] == 'CB', :][:4000].describe()

In [ ]:
CB_describe

In [ ]:
df_sum[df_sum['mark']=='CB']

In [ ]:
df_auto = df_sum.copy()

In [ ]:
cell_bc_auto, _threshold_auto = auto_cell(df_auto)

In [ ]:
df_auto.loc[:, 'mark'] = 'UB'
df_auto.loc[df_sum.index.isin(cell_bc_auto), 'mark'] = 'CB'
CB_describe_auto = df_auto.loc[df_auto['mark'] == 'CB', :].describe()

In [ ]:
CB_describe_auto

In [ ]:
len(set(cell_bc_auto).intersection(set(cell_bc)))

In [ ]:
_threshold_auto

In [ ]:
df_auto[df_auto['mark']=='CB']

In [ ]:
df_sum[df_sum['mark']=='CB']

In [ ]:
_threshold

In [ ]:
target_median_gene = int(CB_describe_auto.loc['50%', 'geneID']) * 0.8

In [ ]:
target_median_gene

In [ ]:
df_sum

In [ ]:
df_sum.loc[df_sum['mark'] == 'CB', :][:4712].describe().loc['50%', 'geneID']

In [ ]:
def search_target_gene_bc():
    target_median_gene = auto_CB_describe(df_sum)[-1] * 0.8
    low = len(cell_bc_auto)
    high = len(cell_bc)
    while low <= high:
        mid = (low + high) // 2
        current_median_gene = df_sum.loc[df_sum['mark'] == 'CB', :][:mid].describe().loc['50%', 'geneID']
        if current_median_gene == target_median_gene:
            return current_median_gene
        elif current_median_gene > target_median_gene:
            high = mid - 1
        else:
            low = mid + 1
    return current_median_gene

In [ ]:
df_sum.loc[df_sum['mark'] == 'CB', :][:4712].reset_index().Barcode

In [ ]:
df_sum.iloc[:4712].reset_index().Barcode.tolist()

In [130]:
df_sum

,readcount,UMI2,UMI,geneID,mark
Barcode,,,,,
AACAACCAGTCGTAGACACCTTAC,210001,181881,76645,6778,CB
GTACGCAAGCCAAGACCGAACTTA,189400,176090,52942,2487,CB
ACTATGCAAACCGAGACCGACAAC,163141,146897,52730,3240,CB
CCTCTATCCAATGGAAGACAGTGC,167824,153922,50412,2548,CB
TGGAACAACCTCCTGAACGTATCA,162767,148077,50098,2794,CB
...,...,...,...,...,...
TAGGATGAACTATGCAAACCGAGA,1,0,1,1,UB
AGTCACTAGAATCTGACGACACAC,1,0,1,1,UB
ACATTGGCCTCAATGATTCACGCA,1,0,1,1,UB


In [132]:
df_sum

,readcount,UMI2,UMI,geneID,mark
Barcode,,,,,
AACAACCAGTCGTAGACACCTTAC,210001,181881,76645,6778,CB
GTACGCAAGCCAAGACCGAACTTA,189400,176090,52942,2487,CB
ACTATGCAAACCGAGACCGACAAC,163141,146897,52730,3240,CB
CCTCTATCCAATGGAAGACAGTGC,167824,153922,50412,2548,CB
TGGAACAACCTCCTGAACGTATCA,162767,148077,50098,2794,CB
...,...,...,...,...,...
TAGGATGAACTATGCAAACCGAGA,1,0,1,1,UB
AGTCACTAGAATCTGACGACACAC,1,0,1,1,UB
ACATTGGCCTCAATGATTCACGCA,1,0,1,1,UB


In [ ]:
idx=6000
int(df_sum.iloc[idx - 1, df_sum.columns == 'UMI'])

In [ ]:
        df_sum.loc[:, 'mark'] = 'UB'
        df_sum.loc[df_sum.index.isin(cell_bc), 'mark'] = 'CB'
        df_sum.to_csv('/SGRNJ06/randd/USER/cjj/TESTDATA/git/celescope_test_script/rna/test1/05.count/test.txt', sep='\t')